<a href="https://colab.research.google.com/github/HumnaArshad19/Assesment_PDA/blob/main/ST20317370_StreamlitApp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Streamlit App - AQI Prediction**

# **St20317370 _ Humna Arshad**




In [1]:
!pip -q install streamlit==1.32.2 joblib scikit-learn pandas numpy matplotlib plotly cloudflared

In [2]:
import os, textwrap
from pathlib import Path

PROJECT = Path("/content/AQI_Streamlit_App")
PAGES = PROJECT / "pages"
PROJECT.mkdir(parents=True, exist_ok=True)
PAGES.mkdir(parents=True, exist_ok=True)

# ---------------- app.py ----------------
app_py = r"""
import streamlit as st

st.set_page_config(page_title="AQI Prediction App", page_icon="🌫️", layout="wide")

# ---- Minimal CSS polish (safe for Streamlit) ----
st.markdown(
    '''
    <style>
      .big-title {font-size: 46px; font-weight: 800; margin-bottom: 0px;}
      .subtle {opacity: 0.8; font-size: 16px; margin-top: 4px;}
      .card {
        padding: 18px 18px;
        border-radius: 16px;
        background: rgba(255,255,255,0.03);
        border: 1px solid rgba(255,255,255,0.08);
      }
      .pill {
        display:inline-block; padding: 6px 12px; border-radius: 999px;
        border: 1px solid rgba(255,255,255,0.15);
        font-size: 14px;
      }
    </style>
    ''',
    unsafe_allow_html=True,
)

st.markdown('<div class="big-title">🌫️ AQI Prediction App</div>', unsafe_allow_html=True)
st.markdown('<div class="subtle">Predict Air Quality Index (AQI) from pollutant concentrations along with Time City Season context.</div>', unsafe_allow_html=True)

st.write("")
c1, c2, c3 = st.columns(3)
with c1:
    st.markdown('<div class="card"><b>✅ Page 1: Overview</b><br/>Project objective → workflow → AQI guide.</div>', unsafe_allow_html=True)
with c2:
    st.markdown('<div class="card"><b>🧪 Page 2: AQI Prediction</b><br/>Enter inputs → Predict AQI + category + health advice.</div>', unsafe_allow_html=True)
with c3:
    st.markdown('<div class="card"><b>📊 Page 3: Model Insights</b><br/>Metrics + plots + feature importance + downloads.</div>', unsafe_allow_html=True)

st.write("")
st.info("Open **AQI Prediction** from the sidebar and try a preset (Good/Moderate/Poor) for quick testing.")
"""


# ---------------- pages/1_Overview.py ----------------
overview_py = r"""
import streamlit as st

st.set_page_config(page_title="Overview", page_icon="📌", layout="wide")

st.title("📌 AQI Prediction Project Overview")
st.caption("A Streamlit ML app to predict AQI using pollutants along with Date, City and Season context.")

st.write("")
left, right = st.columns([1.25, 1])

with left:
    st.subheader("🎯 Objective")
    st.write(
        '''
Build a machine learning model to predict **Air Quality Index (AQI)** using:

- **Pollutants:** PM2.5, PM10, NO, NO2, NOx, NH3, CO, SO2, O3, Benzene, Toluene, Xylene
- **Time features:** Year, Month, Day
- **Context:** City, Season
'''
    )

    st.subheader("✅ How to use the app")
    st.markdown(
        '''
1. Open **AQI Prediction**
2. Use a **preset** or enter your own values
3. Click **Predict AQI ✅**
4. See the predicted AQI, category and health message
5. Open **Model Insights** to view evaluation plots and metrics
'''
    )

    st.subheader("🧠 What the model learns")
    st.write(
        '''
The model learns patterns between pollutant levels and AQI.

Typical relationship:
- Higher **PM2.5 / PM10 / NOx / CO** → higher AQI
- **Season And City** matter due to weather and local pollution sources
'''
    )

def aqi_category(aqi: float):
    # Simple common category guide used for display
    if aqi <= 50:   return "Good ✅", "Safe for everyone"
    if aqi <= 100:  return "Satisfactory 🙂", "Minor discomfort to sensitive people"
    if aqi <= 200:  return "Moderate ⚠️", "Discomfort to people with lung/heart issues"
    if aqi <= 300:  return "Poor ❗", "Breathing discomfort to many people"
    if aqi <= 400:  return "Very Poor 🚨", "Serious health effects"
    return "Severe ☠️", "Emergency conditions"

with right:
    st.subheader("📊 AQI Category Guide")
    sample = [25, 80, 150, 250, 350, 450]
    rows = []
    for v in sample:
        c, msg = aqi_category(v)
        rows.append({"AQI Example": v, "Category": c, "Meaning": msg})
    st.dataframe(rows, use_container_width=True, hide_index=True)

st.divider()

st.subheader("🛠️ Project Workflow")
st.markdown(
    '''
1. **Data Cleaning & Preparation**
   - Handled missing values
   - Created time-based features

2. **EDA**
   - Distributions, Correlations, Seasonal/City Patterns

3. **Model Building**
   - Trained multiple models
   - Selected best model (GridSearch-tuned Random Forest Regressor)
   - Saved final pipeline as: **aqi_prediction_model.joblib**

4. **Deployment**
   - Streamlit multi-page app (Prediction + Insights)
'''
)

st.success("✅ Next: Open **AQI Prediction** page to enter values and predict AQI.")
"""

# ---------------- pages/2_AQI_Prediction.py ----------------
pred_py = r"""
import streamlit as st
import joblib
import pandas as pd
from pathlib import Path
import numpy as np
import calendar

st.set_page_config(page_title="AQI Prediction", page_icon="🧪", layout="wide")
st.title("🧪 AQI Prediction")
st.caption("Enter feature values then click Predict to get AQI output, category and health guidance.")

BASE_DIR = Path(__file__).resolve().parents[1]
MODEL_PATH = BASE_DIR / "aqi_prediction_model.joblib"

@st.cache_resource
def load_model():
    return joblib.load(MODEL_PATH)

# ---- load model ----
if not MODEL_PATH.exists():
    st.error("❌ Model file not found: `aqi_prediction_model.joblib`")
    st.info(f"Put it here: `{MODEL_PATH}`")
    st.stop()

try:
    model = load_model()
except Exception as e:
    st.error("❌ Model failed to load (version mismatch or corrupted file).")
    st.code(str(e))
    st.stop()

# ---- Helpers ----
def aqi_category(aqi: float):
    # category, color, message
    if aqi <= 50:
        return "Good ✅", "#2ecc71", "Safe for everyone. Enjoy normal outdoor activities."
    if aqi <= 100:
        return "Satisfactory 🙂", "#27ae60", "Sensitive people may feel minor discomfort."
    if aqi <= 200:
        return "Moderate ⚠️", "#f1c40f", "People with lung/heart issues should reduce long outdoor exertion."
    if aqi <= 300:
        return "Poor ❗", "#e67e22", "Avoid long outdoor activities. Consider a mask if needed."
    if aqi <= 400:
        return "Very Poor 🚨", "#e74c3c", "Serious health effects possible. Stay indoors if you can."
    return "Severe ☠️", "#c0392b", "Emergency conditions. Avoid outdoor exposure."

def clamp_nonneg(x):
    return float(max(0.0, x))

# ---- Sidebar controls ----
st.sidebar.header("⚙️ Options")
show_input_df = st.sidebar.toggle("Show input table", value=True)
use_presets = st.sidebar.toggle("Show quick presets", value=True)

# ---- Presets ----
preset_map = {
    "Good (low pollution)": dict(pm25=15, pm10=30, no=5, no2=10, nox=15, nh3=10, co=0.6, so2=8, o3=20, benzene=0.5, toluene=1, xylene=0.5),
    "Moderate": dict(pm25=55, pm10=110, no=12, no2=25, nox=35, nh3=18, co=1.1, so2=12, o3=30, benzene=2, toluene=5, xylene=1),
    "Poor (high pollution)": dict(pm25=180, pm10=260, no=30, no2=60, nox=90, nh3=40, co=2.5, so2=25, o3=45, benzene=6, toluene=15, xylene=4),
}

if "preset_choice" not in st.session_state:
    st.session_state.preset_choice = "Moderate"

# ---- UI Form ----
with st.form("aqi_form"):
    top1, top2, top3, top4 = st.columns([1.2, 1, 1, 1])
    with top1:
        city = st.text_input("City", value="Ahmedabad")
    with top2:
        season = st.selectbox("Season", ["Winter", "Summer", "Spring", "Fall"], index=0)
    with top3:
        year = st.number_input("Year", min_value=2015, max_value=2035, value=2019)
    with top4:
        if use_presets:
            st.session_state.preset_choice = st.selectbox("Quick preset", list(preset_map.keys()), index=list(preset_map.keys()).index(st.session_state.preset_choice))
        else:
            st.write("")

    # Month/day with correct max day
    colm, cold = st.columns(2)
    with colm:
        month = st.slider("Month", 1, 12, 7)
    with cold:
        max_day = calendar.monthrange(int(year), int(month))[1]
        day = st.slider("Day", 1, max_day, min(15, max_day))

    st.subheader("Pollutants (enter non-negative values)")
    p = preset_map[st.session_state.preset_choice]

    c1, c2, c3 = st.columns(3)
    with c1:
        pm25 = st.number_input("PM2.5", value=float(p["pm25"]), min_value=0.0)
        pm10 = st.number_input("PM10", value=float(p["pm10"]), min_value=0.0)
        no = st.number_input("NO", value=float(p["no"]), min_value=0.0)
        no2 = st.number_input("NO2", value=float(p["no2"]), min_value=0.0)
    with c2:
        nox = st.number_input("NOx", value=float(p["nox"]), min_value=0.0)
        nh3 = st.number_input("NH3", value=float(p["nh3"]), min_value=0.0)
        co = st.number_input("CO", value=float(p["co"]), min_value=0.0)
        so2 = st.number_input("SO2", value=float(p["so2"]), min_value=0.0)
    with c3:
        o3 = st.number_input("O3", value=float(p["o3"]), min_value=0.0)
        benzene = st.number_input("Benzene", value=float(p["benzene"]), min_value=0.0)
        toluene = st.number_input("Toluene", value=float(p["toluene"]), min_value=0.0)
        xylene = st.number_input("Xylene", value=float(p["xylene"]), min_value=0.0)

    submitted = st.form_submit_button("Predict AQI ✅")

# ---- Build input df ----
input_df = pd.DataFrame([{
    "City": city.strip(),
    "Season": season,
    "Year": int(year),
    "Month": int(month),
    "Day": int(day),
    "PM2.5": clamp_nonneg(pm25),
    "PM10": clamp_nonneg(pm10),
    "NO": clamp_nonneg(no),
    "NO2": clamp_nonneg(no2),
    "NOx": clamp_nonneg(nox),
    "NH3": clamp_nonneg(nh3),
    "CO": clamp_nonneg(co),
    "SO2": clamp_nonneg(so2),
    "O3": clamp_nonneg(o3),
    "Benzene": clamp_nonneg(benzene),
    "Toluene": clamp_nonneg(toluene),
    "Xylene": clamp_nonneg(xylene)
}])

if show_input_df:
    st.write("### Your Input")
    st.dataframe(input_df, use_container_width=True)

# ---- Predict ----
if submitted:
    with st.spinner("Predicting..."):
        try:
            pred = float(model.predict(input_df)[0])
            cat, color, msg = aqi_category(pred)

            st.markdown(
                f'''
                <div style="padding:18px;border-radius:16px;border:1px solid rgba(255,255,255,0.12);">
                  <div style="font-size:18px;opacity:0.9;">Predicted AQI</div>
                  <div style="font-size:44px;font-weight:800;margin-top:4px;">{pred:.2f}</div>
                  <div style="margin-top:8px;">
                    <span style="display:inline-block;padding:6px 12px;border-radius:999px;background:{color};color:white;font-weight:700;">
                      {cat}
                    </span>
                  </div>
                  <div style="margin-top:12px;opacity:0.9;">{msg}</div>
                </div>
                ''',
                unsafe_allow_html=True
            )

        except Exception as e:
            st.error("❌ Prediction failed (feature mismatch with training pipeline).")
            st.write("Most common cause: training pipeline expected different feature names or encodings.")
            st.code(str(e))
"""

# ---------------- pages/3_Model_Insights.py ----------------
insights_py = r"""
import streamlit as st
import joblib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import io

st.set_page_config(page_title="Model Insights", page_icon="📊", layout="wide")
st.title("📊 Model Insights")
st.caption("Evaluation Plots + Metrics + Feature Importance.")

BASE_DIR = Path(__file__).resolve().parents[1]
MODEL_PATH = BASE_DIR / "aqi_prediction_model.joblib"
EVAL_PATH = BASE_DIR / "eval_artifact.joblib"

@st.cache_resource
def load_model():
    return joblib.load(MODEL_PATH)

@st.cache_data
def load_eval():
    return joblib.load(EVAL_PATH)

# ---- load model ----
if not MODEL_PATH.exists():
    st.error("❌ Model file not found: `aqi_prediction_model.joblib`")
    st.stop()

try:
    model = load_model()
except Exception as e:
    st.error("❌ Model failed to load.")
    st.code(str(e))
    st.stop()

tabs = st.tabs(["📌 Metrics", "📈 Plots", "🧠 Feature Importance", "⬇️ Downloads"])

# ---- Load eval if present ----
eval_ok = EVAL_PATH.exists()
if eval_ok:
    try:
        eval_data = load_eval()
        y_true = np.array(eval_data["y_true"]).reshape(-1)
        y_pred = np.array(eval_data["y_pred"]).reshape(-1)
        residuals = y_true - y_pred
    except Exception as e:
        eval_ok = False
        err = str(e)

# ---------------- Metrics tab ----------------
with tabs[0]:
    st.subheader("Model Metrics")
    if not eval_ok:
        st.warning("⚠️ `eval_artifact.joblib` not found or invalid. Metrics require y_true and y_pred.")
        if "err" in locals():
            st.code(err)
        st.info(f"Expected file path: `{EVAL_PATH}`")
    else:
        mae = float(np.mean(np.abs(residuals)))
        rmse = float(np.sqrt(np.mean(residuals**2)))
        bias = float(np.mean(residuals))

        c1, c2, c3, c4 = st.columns(4)
        c1.metric("MAE", f"{mae:.2f}")
        c2.metric("RMSE", f"{rmse:.2f}")
        c3.metric("Bias (mean residual)", f"{bias:.2f}")
        c4.metric("Samples", f"{len(y_true)}")



# ---------------- Plots tab ----------------
with tabs[1]:
    st.subheader("Evaluation Plots")
    if not eval_ok:
        st.warning("⚠️ Add `eval_artifact.joblib` to see plots.")
    else:
        colA, colB = st.columns(2)

        with colA:
            st.markdown("### Predicted vs Actual")
            fig = plt.figure(figsize=(7, 5))
            plt.scatter(y_true, y_pred, alpha=0.5)
            mn = float(min(y_true.min(), y_pred.min()))
            mx = float(max(y_true.max(), y_pred.max()))
            plt.plot([mn, mx], [mn, mx])
            plt.xlabel("Actual AQI")
            plt.ylabel("Predicted AQI")
            plt.title("Predicted vs Actual AQI")
            plt.tight_layout()
            st.pyplot(fig)

        with colB:
            st.markdown("### Residuals vs Predicted")
            fig = plt.figure(figsize=(7, 5))
            plt.scatter(y_pred, residuals, alpha=0.5)
            plt.axhline(0)
            plt.xlabel("Predicted AQI")
            plt.ylabel("Residual (Actual - Predicted)")
            plt.title("Residual Plot")
            plt.tight_layout()
            st.pyplot(fig)

        st.markdown("### Distribution: Actual vs Predicted")
        fig = plt.figure(figsize=(10, 5))
        plt.hist(y_true, bins=30, alpha=0.6, label="Actual")
        plt.hist(y_pred, bins=30, alpha=0.6, label="Predicted")
        plt.xlabel("AQI")
        plt.ylabel("Count")
        plt.title("Distribution of Actual vs Predicted AQI")
        plt.legend()
        plt.tight_layout()
        st.pyplot(fig)

# ---------------- Feature Importance tab ----------------
with tabs[2]:
    st.subheader("Feature Importance ")
    try:
        rf = getattr(model, "named_steps", {}).get("model", None)
        prep = getattr(model, "named_steps", {}).get("prep", None)

        if rf is None or not hasattr(rf, "feature_importances_") or prep is None:
            st.info("This pipeline does not expose feature_importances_. (That’s okay.)")
        else:
            importances = rf.feature_importances_

            num_features = prep.transformers_[0][2]
            ohe = prep.named_transformers_["cat"].named_steps["encoder"]
            cat_features = ohe.get_feature_names_out(["City", "Season"])
            all_features = list(num_features) + list(cat_features)

            fi = pd.DataFrame({"feature": all_features, "importance": importances})
            fi = fi.sort_values("importance", ascending=False).head(20)

            st.dataframe(fi, use_container_width=True)

            fig = plt.figure(figsize=(10, 6))
            plt.barh(fi["feature"][::-1], fi["importance"][::-1])
            plt.xlabel("Importance")
            plt.title("Top 20 Feature Importances")
            plt.tight_layout()
            st.pyplot(fig)

            st.markdown(
                '''
**Interpretation**
- Bigger bars → model relies more on that feature.
- Pollutants like PM2.5/PM10 often dominate AQI predictions.
- City/Season features matter due to geographic and weather differences.
'''
            )

    except Exception as e:
        st.warning("Feature importance failed to compute.")
        st.code(str(e))

# ---------------- Downloads tab ----------------
with tabs[3]:
    st.subheader("Downloads")
    if not eval_ok:
        st.warning("⚠️ Add eval artifact to enable downloads.")
    else:
        df_out = pd.DataFrame({"y_true": y_true, "y_pred": y_pred, "residual": residuals})
        csv_bytes = df_out.to_csv(index=False).encode("utf-8")
        st.download_button("Download predictions CSV", data=csv_bytes, file_name="aqi_predictions.csv", mime="text/csv")

        st.caption("Download this csv file of model prediction of city.")
"""

# write files
(PROJECT / "app.py").write_text(textwrap.dedent(app_py).strip() + "\n", encoding="utf-8")
(PAGES / "1_Overview.py").write_text(textwrap.dedent(overview_py).strip() + "\n", encoding="utf-8")
(PAGES / "2_AQI_Prediction.py").write_text(textwrap.dedent(pred_py).strip() + "\n", encoding="utf-8")
(PAGES / "3_Model_Insights.py").write_text(textwrap.dedent(insights_py).strip() + "\n", encoding="utf-8")

print("✅ Enhanced Streamlit app updated at:", PROJECT)
print("➡️ Model path:", PROJECT / "aqi_prediction_model.joblib")
print("➡️ Eval path :", PROJECT / "eval_artifact.joblib")


✅ Enhanced Streamlit app updated at: /content/AQI_Streamlit_App
➡️ Model path: /content/AQI_Streamlit_App/aqi_prediction_model.joblib
➡️ Eval path : /content/AQI_Streamlit_App/eval_artifact.joblib


In [3]:
#  source path
#!cp "/content/AQI_Streamlit_App/eval_artifact.joblib" "/content/AQI_Streamlit_App/aqi_prediction_model.joblib"
!ls -lah /content/AQI_Streamlit_App


total 138M
drwxr-xr-x 4 root root 4.0K Jan 15 05:24 .
drwxr-xr-x 1 root root 4.0K Jan 15 02:44 ..
-rw-r--r-- 1 root root 1.7K Jan 15 05:27 app.py
-rw-r--r-- 1 root root 138M Jan 15 05:25 aqi_prediction_model.joblib
-rw-r--r-- 1 root root  93K Jan 15 05:24 eval_artifact.joblib
drwxr-xr-x 2 root root 4.0K Jan 15 05:23 .ipynb_checkpoints
drwxr-xr-x 2 root root 4.0K Jan 15 02:29 pages


In [4]:
!tail -n 80 /content/streamlit.log
!tail -n 80 /content/cloudflared.log




  You can now view your Streamlit app in your browser.

  URL: http://0.0.0.0:8501


  A new version of Streamlit is available.

  See what's new at https://discuss.streamlit.io/c/announcements

  Enter the following command to upgrade:
  $ pip install streamlit --upgrade

  Stopping...
2026-01-15T05:01:32Z INF Thank you for trying Cloudflare Tunnel. Doing so, without a Cloudflare account, is a quick way to experiment and try it out. However, be aware that these account-less Tunnels have no uptime guarantee, are subject to the Cloudflare Online Services Terms of Use (https://www.cloudflare.com/website-terms/), and Cloudflare reserves the right to investigate your use of Tunnels for violations of such terms. If you intend to use Tunnels in production you should use a pre-created named tunnel by following: https://developers.cloudflare.com/cloudflare-one/connections/connect-apps
2026-01-15T05:01:32Z INF Requesting new quick Tunnel on trycloudflare.com...
2026-01-15T05:01:37Z INF +----

In [5]:
!pkill -f cloudflared || true
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 -O /content/cloudflared
!chmod +x /content/cloudflared

!nohup /content/cloudflared tunnel --url http://127.0.0.1:8501 > /content/cloudflared.log 2>&1 &
!sleep 3
!cat /content/cloudflared.log | grep -Eo "https://[-a-zA-Z0-9.]*trycloudflare.com" | head -n 1


^C


In [6]:
# ===============================
# COLAB STREAMLIT + CLOUDFLARE
# ===============================

import os, time, subprocess, re

# 1️⃣ Go to project folder
os.chdir("/content/AQI_Streamlit_App")

# 2️⃣ Kill old processes
subprocess.run("pkill -f streamlit || true", shell=True)
subprocess.run("pkill -f cloudflared || true", shell=True)
subprocess.run("fuser -k 8501/tcp || true", shell=True)

# 3️⃣ Installing compatible libraries
subprocess.run(
    "pip install -q streamlit==1.32.2 joblib pandas numpy scikit-learn matplotlib",
    shell=True
)

# 4️⃣ Start Streamlit in background
subprocess.Popen(
    "nohup streamlit run app.py "
    "--server.port 8501 "
    "--server.address 0.0.0.0 "
    "--server.enableCORS false "
    "--server.enableXsrfProtection false "
    "> /content/streamlit.log 2>&1 &",
    shell=True
)

# 5️⃣ Wait until Streamlit is LIVE
print("⏳ Waiting for Streamlit to start...")
for i in range(30):
    try:
        import urllib.request
        urllib.request.urlopen("http://127.0.0.1:8501", timeout=1)
        print("✅ Streamlit is running")
        break
    except:
        time.sleep(1)
else:
    print("❌ Streamlit failed. Showing logs:")
    os.system("tail -n 50 /content/streamlit.log")
    raise SystemExit

# 6️⃣ Start Cloudflare tunnel
subprocess.Popen(
    "nohup /content/cloudflared tunnel "
    "--url http://127.0.0.1:8501 "
    "> /content/cloudflared.log 2>&1 &",
    shell=True
)

# 7️⃣ Extract public URL
print("⏳ Creating public URL...")
public_url = None
for i in range(30):
    with open("/content/cloudflared.log", "r") as f:
        text = f.read()
    match = re.search(r"https://[a-zA-Z0-9-]+\.trycloudflare\.com", text)
    if match:
        public_url = match.group(0)
        break
    time.sleep(1)

if public_url:
    print("\n🎉 YOOO STREAMLIT APP IS LIVE:")
    print(public_url)
else:
    print("❌ Could not get Cloudflare URL")
    os.system("tail -n 50 /content/cloudflared.log")


⏳ Waiting for Streamlit to start...
✅ Streamlit is running
⏳ Creating public URL...

🎉 YOOO STREAMLIT APP IS LIVE:
https://stats-benz-breaking-punch.trycloudflare.com


In [7]:
import joblib
obj = joblib.load("/content/AQI_Streamlit_App/aqi_prediction_model.joblib")
print(type(obj))

if isinstance(obj, dict):
    print("Keys:", obj.keys())


<class 'sklearn.pipeline.Pipeline'>


Here is my recording of Running streamlit App:

https://outlookuwicac-my.sharepoint.com/:v:/g/personal/st20317370_outlook_cardiffmet_ac_uk/IQCGIDNydckkS53H19a87H5hAXemXKLEChlSzYS-8_vmzYY?nav=eyJyZWZlcnJhbEluZm8iOnsicmVmZXJyYWxBcHAiOiJTdHJlYW1XZWJBcHAiLCJyZWZlcnJhbFZpZXciOiJTaGFyZURpYWxvZy1MaW5rIiwicmVmZXJyYWxBcHBQbGF0Zm9ybSI6IldlYiIsInJlZmVycmFsTW9kZSI6InZpZXcifX0%3D&e=wOkGpE

In [13]:
!zip -r AQI_Streamlit_App.zip AQI_Streamlit_App


	zip warning: name not matched: AQI_Streamlit_App

zip error: Nothing to do! (try: zip -r AQI_Streamlit_App.zip . -i AQI_Streamlit_App)


In [14]:
!pwd


/root


In [15]:
!zip -r /content/AQI_Streamlit_App.zip /content/AQI_Streamlit_App


  adding: content/AQI_Streamlit_App/ (stored 0%)
  adding: content/AQI_Streamlit_App/aqi_prediction_model.joblib (deflated 74%)
  adding: content/AQI_Streamlit_App/.ipynb_checkpoints/ (stored 0%)
  adding: content/AQI_Streamlit_App/pages/ (stored 0%)
  adding: content/AQI_Streamlit_App/pages/1_Overview.py (deflated 49%)
  adding: content/AQI_Streamlit_App/pages/3_Model_Insights.py (deflated 65%)
  adding: content/AQI_Streamlit_App/pages/2_AQI_Prediction.py (deflated 62%)
  adding: content/AQI_Streamlit_App/eval_artifact.joblib (deflated 41%)
  adding: content/AQI_Streamlit_App/app.py (deflated 52%)


In [17]:
from google.colab import files
files.download("/content/AQI_Streamlit_App.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>